<a href="https://colab.research.google.com/github/KENTAROSZK/unsloth_test/blob/main/unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 参考ページ：
### https://zenn.dev/the_exile/articles/unsloth-llama3-fine-tuning
### https://qiita.com/Maki-HamarukiLab/items/891d03e570be9fc8d13b
### https://qiita.com/Maki-HamarukiLab/items/a577d05cd224052378df

!pip install unsloth
!pip install wandb

## wandbの設定

In [2]:
import wandb
from google.colab import userdata
wandb_api_key = userdata.get('WANDB_API_KEY')
!wandb login $wandb_api_key

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## モデルの準備

In [3]:
# モデルの準備

from unsloth import FastLanguageModel
import torch


# 基本設定
max_seq_length = 2048  # RoPEスケーリングで自動調整
dtype = None  # GPU種別に応じて自動選択
load_in_4bit = True  # メモリ効率化のための4bit量子化


# モデルとトークナイザーの読み込み
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## ファインチューニングする前に質問してみる。

In [4]:
# 高速推論の有効化
FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": "フィボナッチ数列を続けてください: 1, 1, 2, 3, 5, 8,"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    inputs,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=1024,
    temperature=0.5,
)

llm_answer = tokenizer.decode(
    outputs[0],
    skip_special_tokens = True
)

print(llm_answer)

system

Cutting Knowledge Date: December 2023
Today Date: 01 Jan 2025

user

フィボナッチ数列を続けてください: 1, 1, 2, 3, 5, 8,assistant

フィボナッチ数列は、各項が前の 2 項の合計として得られる数列です。次の項は次のようになります。

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,...

フィボナッチ数列は、自然数の素数の分布や、自然数の平方数の数の分布など、さまざまな数学的問題に使用されることが多いです。


## LoRAアダプターの追加

効率的にチューニングするためにLoRAアダプターで学習する。

In [11]:
# LoRAアダプターの追加
# パラメータの効率的な更新のため、LoRAアダプターを追加します。

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRAアダプターのランク（推奨値: 8, 16, 32, 64, 128）
    target_modules = [ # ファインチューニングするすべてのモジュールを選択します。メモリ使用量を削減してトレーニングを高速化するために、いくつか削除することもできますが、お勧めしません。すべてのモジュールでトレーニングしてください。
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16, # ファインチューニングのスケーリング係数。数値が大きいほど、ファインチューニングはデータセットについて多く学習しますが、オーバーフィッティングを促進する可能性があります。ランクrと同じか、2倍にすることをお勧めします。
    lora_dropout = 0, # トレーニングを高速化するために、これを0のままにします。オーバーフィッティングを減らすことはできますが、それほどではありません。
    bias = "none", # トレーニングを高速化し、オーバーフィッティングを減らすために、これを0のままにします。
    use_gradient_checkpointing = "unsloth", # オプションは、True、False、"unsloth"です。"unsloth"を推奨します。これは、メモリ使用量をさらに30%削減し、非常に長いコンテキストのファインチューニングをサポートしているためです。詳細については、https://unsloth.ai/blog/long-context をご覧ください。
    random_state = 3407,
    use_rslora=False,  # ランク安定化LoRAをサポート。lora_alpha = 16を自動的に設定する高度な機能です。必要に応じて使用できます。
    loftq_config = None,
)


Unsloth: Already have LoRA adapters! We shall skip this step.


## データの準備

注意：ファインチューニングするには`dataset`形式になっている必要がある。

In [6]:
# https://github.com/the-exile-110/unsloth-llama3-fine-tuning/blob/main/dataset.json
json_data =[
  {
    "instruction": "心臓が一つだけ残っても生きられますか？",
    "input": "",
    "output": "はい、人間はもともと心臓が一つしかありません。"
  },
  {
    "instruction": "もうすぐ水泳の授業があるのに、昨日洗った水着がまだ乾いていません。どうすればいいですか？",
    "input": "",
    "output": "水泳中はどうせ水着が濡れるので、乾かす必要はありません。"
  },
  {
    "instruction": "一度しか生まれていないのに、なぜ毎年誕生日を祝うのですか？",
    "input": "",
    "output": "誕生日を祝うのは、生まれた日を祝うのではなく、毎年の成長と経験を記念し、未来の一年に期待と祝福を込めるためです。"
  },
  {
    "instruction": "私は25歳で父は50歳です。つまり、父は2歳で私を生んだのですか？",
    "input": "",
    "output": "違います。あなたが25歳で、父が50歳なら、父はあなたが生まれたとき25歳だったということです。"
  },
  {
    "instruction": "5000円で新車を買うなら何がいいですか？",
    "input": "",
    "output": "どうしても答えるなら、おもちゃの車しか買えないでしょう。"
  },
  {
    "instruction": "酸素と水素を同時に吸入したら、水を飲んでいることになるのですか？",
    "input": "",
    "output": "違います。水は水素と酸素で構成されていますが、単に酸素と水素を吸入するだけでは水は形成されません。水の形成には特定の条件下で化学反応が必要で、その反応は激しく、大量の熱を発生します。人体にとって非常に危険ですので、酸素と水素を吸入して水を飲むことはできません。"
  },
  {
    "instruction": "赤壁の戦いで、曹操はなぜ119に電話して救助を求めなかったのですか？",
    "input": "",
    "output": "三国時代には電話や現代の消防システムがなかったので、曹操は119に電話して救助を求めることはできませんでした。"
  },
  {
    "instruction": "一昨日は誕生日おめでとうと言われたのに、今日はなぜ言われないのですか？",
    "input": "",
    "output": "今日はあなたの誕生日ではなく、一昨日があなたの誕生日だったからです。"
  },
  {
    "instruction": "Bluetoothイヤホンが壊れたのですが、病院の歯科か耳科に行くべきですか？",
    "input": "",
    "output": "Bluetoothイヤホンが壊れた場合、修理店や専門の電子製品修理技術者に頼むべきで、病院の歯科や耳科に行くべきではありません。"
  },
  {
    "instruction": "1キロの綿と1キロの鉄が同時に水に落ちたら、どちらを先に救いますか？",
    "input": "",
    "output": "実際には、鉄は水に沈み、綿は水面に浮くので、救う必要はありません。どちらも救う必要はありません。"
  },
  {
    "instruction": "刑務所には全て罪人がいるのに、なぜ警察は刑務所に行って捕まえないのですか？",
    "input": "",
    "output": "彼らはすでに捕まって刑務所にいるので、再び捕まえる必要はありません。"
  },
  {
    "instruction": "人間には左耳と右耳しかないのに、なぜ中耳炎になるのですか？",
    "input": "",
    "output": "中耳炎は「真ん中の耳」に発生するのではなく、耳の構造の中耳部分に発生する炎症です。耳の構造は外耳、中耳、内耳の三つに分かれており、中耳炎は中耳部分が炎症を起こすことです。つまり、人間には左耳と右耳しかなくても中耳炎になることがあります。"
  }
]

In [8]:
# ファインチューニングする時用にdatasetを作成する
from datasets import Dataset


# ファインチューニング用にテンプレートプロンプトを用意する
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}
### Response:
{}"""


def formatting_prompt(examples: list) -> dict:
    instructions = [item["instruction"] for item in examples]
    inputs = [item["input"] for item in examples]
    outputs = [item["output"] for item in examples]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    return {"text": texts}


_dataset = formatting_prompt(json_data)
# datasetの型
dataset = Dataset.from_dict(_dataset)

In [9]:
dataset[0]

{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n心臓が一つだけ残っても生きられますか？\n### Input:\n\n### Response:\nはい、人間はもともと心臓が一つしかありません。'}

## モデルの学習

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc=2,
    packing=False,  # 短いシーケンスの場合、トレーニングを5倍高速化できます
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    args = TrainingArguments(
        per_device_train_batch_size = 2,# GPUのメモリをより多く活用したい場合は、バッチサイズを増やします。また、トレーニングをよりスムーズにし、オーバーフィットを防ぐためにも、これを増やします。ただし、パディングの問題によりトレーニングが実際に遅くなる可能性があるため、通常はお勧めしません。代わりに、gradient_accumulation_stepsを増やすことをお勧めします。
        gradient_accumulation_steps = 4,#上記のバッチサイズを増やすことと同じ効果がありますが、メモリ消費量には影響しません。通常、トレーニング損失曲線をよりスムーズにしたい場合は、これを増やすことをお勧めします。
        warmup_steps = 5,
        max_steps = 60, # トレーニングを高速化するために、ステップを60に設定しました。数時間かかる可能性のある完全なトレーニング実行の場合は、代わりにmax_stepsをコメントアウトし、num_train_epochs = 1に置き換えます。1に設定すると、データセット全体を1回完全にパスします。通常は1〜3パスを推奨します。それ以上は推奨しません。それ以上パスするとオーバーフィットの可能性が高くなります。
        #num_train_epochs=1
        learning_rate = 2e-4, # ファインチューニングプロセスを遅くしたい場合は学習率を下げますが、より高い精度の結果に収束する可能性があります。通常は、試してみる値として、2e-4、1e-4、5e-5、2e-5を推奨します。
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir = "outputs",
    ),
)

# 学習の実行
trainer_stats = trainer.train()


Map (num_proc=2):   0%|          | 0/12 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 12 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,2.858900
2,3.192700
3,2.877100
4,3.103700
5,2.710900
6,2.776300
7,2.311100
8,2.124900
9,1.811000
10,1.588000


## 学習したモデルを使って推論させる

In [25]:
# 高速推論の有効化
FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": "フィボナッチ数列を続けてください: 1, 1, 2, 3, 5, 8,"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    inputs,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=1024,
    temperature=0.3,
)

llm_answer = tokenizer.decode(
    outputs[0],
    skip_special_tokens = True
)

print(llm_answer)

system

Cutting Knowledge Date: December 2023
Today Date: 01 Jan 2025

user

フィボナッチ数列を続けてください: 1, 1, 2, 3, 5, 8,assistant

次の数は8と言われていますが、実際には次の数は8と言われないで、次の数は8と同時に現れるのではなく、次の数は8と同時に現れるのではなく、次の数は8と同時に現れるのではなく、次の数は8の次に現れる数で、実際には9です。

つまり、正しく言えば、1、1、2、3、5、8、9です。


In [24]:
# 高速推論の有効化
FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": "赤壁の戦いで、曹操はなぜ119に電話して救助を求めなかったのですか？"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    inputs,
    pad_token_id=tokenizer.pad_token_id,
    max_new_tokens=1024,
    temperature=0.3,
)

llm_answer = tokenizer.decode(
    outputs[0],
    skip_special_tokens = True
)

print(llm_answer)

system

Cutting Knowledge Date: December 2023
Today Date: 01 Jan 2025

user

赤壁の戦いで、曹操はなぜ119に電話して救助を求めなかったのですか？assistant

曹操は119に電話して救助を求めることはできませんでした。119は現代の消防システムであり、119は119に電話して消防を求めることです。曹操は119に電話して救助を求めることはできませんでしたので、赤壁の戦いでどうしたらいいですか？

実際には、曹操は119に電話して救助を求めるのではなく、現代の消防システムがなかったので、救助を求めるにはどうしたらいいですか？

曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めることです。つまり、曹操は119に電話して救助を求めるのではなく、119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

実際には、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めることです。つまり、曹操は119に電話して救助を求めるのではなく、119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、曹操は119に電話して救助を求めるのではなく、119は119に電話して消防を求めるべきで、119に電話して救助を求めるべきではありません。

つまり、

## LoRAモデルを保存する

In [18]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

## モデルを結合し、GGUF量子化して保存する

In [19]:
model.save_pretrained_gguf(
    "model",
    tokenizer,
    quantization_method = [
        "q4_k_m",
        "q8_0",
        "q5_k_m"
    ]
)

Unsloth: Will remove a cached repo with size 1.4K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.34 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 22.83it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'q8_0', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map 

In [21]:
# 16bit形式でのアップロード
# VLLMなどで使用する場合は16bit形式が便利です。
model.push_to_hub_merged(
    "./model-name-16bit",
    tokenizer,
    save_method = "merged_16bit"
)


RuntimeError: Unsloth: Please supply a token!
Go to https://huggingface.co/settings/tokens

12